In [1]:
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from datetime import date
#import geopandas as gpd

In [2]:
api = SentinelAPI('username', 'password', 'https://scihub.copernicus.eu/dhus/')

In [3]:
import pandas as pd
import os

In [4]:
geo_json_files = os.listdir(path='C:/Users/Administrator/Desktop/satellite_project/map_locations')
geo_json_files

['detroit.geojson',
 'london.geojson',
 'multiple_cities.geojson',
 'singapore.geojson']

In [5]:
folder_path = 'C:/Users/Administrator/Desktop/satellite_project/map_locations/'

In [15]:
footprint = geojson_to_wkt(read_geojson(folder_path+geo_json_files[2]), feature_number=0)

In [9]:
import json

In [17]:
json_data = open(folder_path+'multiple_cities.geojson')
data = json.load(json_data)
print(len(data['features']))
features_lenght = len(data['features'])

6


In [18]:
multiple_footprints = []
for fno in range(features_lenght):
    multiple_footprints.append(geojson_to_wkt(read_geojson(folder_path+geo_json_files[2]), feature_number=fno))

In [19]:
multiple_footprints

['POLYGON((-95.0177 29.4049,-94.9169 29.4049,-94.9169 29.4683,-95.0177 29.4683,-95.0177 29.4049))',
 'POLYGON((-62.5417 46.3941,-62.4367 46.3941,-62.4367 46.4483,-62.5417 46.4483,-62.5417 46.3941))',
 'POLYGON((10.8947 47.7108,11.0054 47.7108,11.0054 47.7643,10.8947 47.7643,10.8947 47.7108))',
 'POLYGON((76.8458 28.6847,76.9034 28.6847,76.9034 28.7192,76.8458 28.7192,76.8458 28.6847))',
 'POLYGON((78.3007 17.4519,78.3608 17.4519,78.3608 17.4875,78.3007 17.4875,78.3007 17.4519))',
 'POLYGON((85.8132 26.7371,85.8286 26.7371,85.8286 26.7463,85.8132 26.7463,85.8132 26.7371))']

In [58]:
def get_product_name(footprints_list):
    product_names_list = []
    for num,fp in enumerate(footprints_list):
        products = api.query(fp,
                     date = ('20190601', '20190809'),
                     platformname = 'Sentinel-2',
                     processinglevel = 'Level-2A',
                     cloudcoverpercentage = (0,10)
                    )
        products_df = api.to_dataframe(products)
        if (len(products_df.columns) != 0):
            print(f'images obtained for {num} indexed footprint')
            products_df.vegetationpercentage.sort_values(ascending=False)
            product_name = products_df.vegetationpercentage.sort_values(ascending=False).keys()[0]
            product_names_list.append(product_name)
        else:
            print(f'no satellite image available for specified date {num} indexed footprint for the given filters')
    return product_names_list

In [59]:
product_list = get_product_name(multiple_footprints)

images obtained for 0 indexed footprint
images obtained for 1 indexed footprint
images obtained for 2 indexed footprint
images obtained for 3 indexed footprint
images obtained for 4 indexed footprint
images obtained for 5 indexed footprint


In [60]:
product_list

['ae5949c0-133a-4586-ae06-0a5f317f5a19',
 'dc22b919-2503-47b6-a3db-34103ed96761',
 '5960b3ca-de73-4ee2-ba78-e0e8394a9e65',
 '90c76759-d6c2-4693-8e60-aae6a355a25a',
 '8800290f-b968-4d2e-bec3-35f1af1f622f',
 '28be1e7d-75aa-47df-be6a-d9780067200e']

In [61]:
for sat_prod in product_list:
    api.download(sat_prod)

Downloading: 100%|██████████████████████████████████████████████████████████████████| 761M/761M [06:46<00:00, 1.87MB/s]
MD5 checksumming: 100%|██████████████████████████████████████████████████████████████| 761M/761M [00:02<00:00, 343MB/s]
Downloading: 100%|██████████████████████████████████████████████████████████████████| 924M/924M [09:36<00:00, 2.64MB/s]
MD5 checksumming: 100%|██████████████████████████████████████████████████████████████| 924M/924M [00:02<00:00, 339MB/s]
Downloading: 100%|███████████████████████████████████████████████████████████████████| 558M/558M [04:55<00:00, 720kB/s]
MD5 checksumming: 100%|██████████████████████████████████████████████████████████████| 558M/558M [00:01<00:00, 348MB/s]
Downloading: 100%|████████████████████████████████████████████████████████████████| 1.18G/1.18G [10:58<00:00, 1.58MB/s]
MD5 checksumming: 100%|████████████████████████████████████████████████████████████| 1.18G/1.18G [00:03<00:00, 343MB/s]
Downloading: 100%|██████████████████████

In [35]:
products_df.vegetationpercentage.sort_values(ascending=False).keys()[0]

f78c266f-c392-49ba-ac39-ec30c9250a2a    86.993003
380f1e95-a296-48b1-ab10-4661c02aa60a    81.513250
7a688a1a-bb58-4483-bc60-d63c52bd0998    80.016559
c3aaa768-75f4-449e-8080-f53ad298f556    79.652047
9238f8a3-3d2c-4a91-884b-92e18b6e7b03    67.849618
5edfb3e1-885b-4948-8417-e4bebfb088bd    67.554456
Name: vegetationpercentage, dtype: float64